In [1]:
from src.augmentations.CustomAugmentations import CustomAugmentationsTF
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
import os
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
from sklearn.metrics import confusion_matrix, accuracy_score
from tqdm.notebook import tqdm


from src.kerasCNN import *
import pickle



import random
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)
os.environ['PYTHONHASHSEED'] = '42'
keras.utils.set_random_seed(42)
# tf.config.experimental.enable_op_determinism()

In [2]:
height,width=32,32
batch_size=64

In [3]:
optimsers = ['adam', 'sgd']
learning_rates = [
    0.0001, 
    0.001, 
    0.01]

augmented = original_dataset.map(lambda image, label: (custom_aug.augment(image), label))



In [4]:
EPOCHS = 40





for optimser in tqdm(optimsers, desc="Optimser"):
    for lr in tqdm(learning_rates, desc="Learning Rate"):
        # if optimiser adam and lr 0.0001 or optimiser adam and lr 0.001 skip
        if optimser == 'adam' or (optimser == 'sgd' and lr in [0.0001]):
            continue

        hyperparam_str = f"{optimser}_{lr}"
        print(f"Hyperparameters: {hyperparam_str}")

        histories = list()
        accuracies = list()
        class_accuracies = list()
        conf_mats = list()

        for iteration in tqdm(range(5), desc="Iteration"):
            train_set = tf.keras.preprocessing.image_dataset_from_directory(
                "../data/CINIC10/train",
                labels="inferred",
                label_mode="int",
                class_names=None,
                color_mode="rgb",
                batch_size=batch_size,
                image_size=(height, width),
                shuffle=True,
                seed=iteration,
                validation_split=0.05,
                subset="training",
            )

            val_set = tf.keras.preprocessing.image_dataset_from_directory(
                "../data/CINIC10/valid",
                labels="inferred",
                label_mode="int",
                class_names=None,
                color_mode="rgb",
                batch_size=batch_size,
                image_size=(height, width),
                shuffle=True,
                seed=iteration,
                validation_split=0.05,
                subset="validation",
            )

            test_set = tf.keras.preprocessing.image_dataset_from_directory(
                "../data/CINIC10/test",
                labels="inferred",
                label_mode="int",
                class_names=None,
                color_mode="rgb",
                batch_size=batch_size,
                image_size=(height, width),
                shuffle=False,
            )

            dnn_model = create_cnn(
                kernel_sizes=[3, 3, 3,],
                num_filters=[32, 64, 64],
                fc_sizes=[128, 64, 64],
            )
        
            print(f"Fitting model {iteration}")
            history = dnn_model.fit(train_set, validation_data=val_set, epochs=EPOCHS)
            
            print(f"Predicting model {iteration}")
            preds = dnn_model.predict(test_set, verbose=2)
            preds = preds.argmax(axis=1)
            classes = test_set.class_names
            test_labels = list()
            for images, labels in test_set:
                class_labels = [int(label) for label in labels]
                test_labels.extend(class_labels)
            test_labels = np.array(test_labels)

            conf_mat = confusion_matrix(test_labels, preds)
            accuracy = accuracy_score(test_labels, preds)
            print(f"Accuracy: {accuracy} for model {iteration}")
            class_accuracy = conf_mat.diagonal() / conf_mat.sum(axis=1)

            print(f"Saving model {iteration}")
            dnn_model.save(f"../models/BESTClassicCNN_hyper_{hyperparam_str}.h5")
            histories.append(deepcopy(history.history))
            accuracies.append(deepcopy(accuracy))
            class_accuracies.append(deepcopy(class_accuracy))
            conf_mats.append(deepcopy(conf_mat))


        with open(f"results/HISTORY_BESTClassicCNN_hyper_{hyperparam_str}.pkl", "wb") as f:
            pickle.dump(histories, f)

        with open(f"results/ACCURACY_BESTClassicCNN_hyper_{hyperparam_str}.pkl", "wb") as f:
            pickle.dump(accuracies, f)

        with open(f"results/CLASS_ACCURACY_BESTClassicCNN_hyper_{hyperparam_str}.pkl", "wb") as f:
            pickle.dump(class_accuracies, f)

        with open(f"results/CONF_MAT_BESTClassicCNN_hyper_{hyperparam_str}.pkl", "wb") as f:
            pickle.dump(conf_mats, f)

Optimser:   0%|          | 0/2 [00:00<?, ?it/s]

Learning Rate:   0%|          | 0/3 [00:00<?, ?it/s]

Learning Rate:   0%|          | 0/3 [00:00<?, ?it/s]

Hyperparameters: sgd_0.001


Iteration:   0%|          | 0/5 [00:00<?, ?it/s]

Found 90000 files belonging to 10 classes.
Using 85500 files for training.
Found 90000 files belonging to 10 classes.
Using 4500 files for validation.
Found 90000 files belonging to 10 classes.
Fitting model 0
Epoch 1/40
1336/1336 [==============================] - 44s 31ms/step - loss: 1.6146 - accuracy: 0.4035 - val_loss: 1.5170 - val_accuracy: 0.4420
Epoch 2/40
1336/1336 [==============================] - 13s 10ms/step - loss: 1.3513 - accuracy: 0.5128 - val_loss: 1.3793 - val_accuracy: 0.4887
Epoch 3/40
1336/1336 [==============================] - 13s 9ms/step - loss: 1.2386 - accuracy: 0.5569 - val_loss: 1.2914 - val_accuracy: 0.5311
Epoch 4/40
1336/1336 [==============================] - 13s 10ms/step - loss: 1.1648 - accuracy: 0.5836 - val_loss: 1.3242 - val_accuracy: 0.5384
Epoch 5/40
1336/1336 [==============================] - 13s 10ms/step - loss: 1.1046 - accuracy: 0.6068 - val_loss: 1.2948 - val_accuracy: 0.5373
Epoch 6/40
1336/1336 [==============================] - 13s 1